In [1]:
import pandas as pd
from datetime import datetime, time

# Load file into dataframe
folder_path = 'c:/Users/aoife/Documents/Project/DataTables/'
save_path = 'C:/Users/aoife/Documents/Project/filtered_data/'



# Body Mass Index

In [2]:
# get the average height/body mass for each participant

df_bm = pd.read_csv(folder_path + 'body-mass.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])
df_h = pd.read_csv(folder_path + 'height.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])

bmi_dfs = [df_bm, df_h]

for df in bmi_dfs:
    if 'value' in df.columns:

        # Convert timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%SZ')
        # Convert timestamp to date

        # Get the average value for each participant
        df['value'] = df.groupby(['participantId'])['value'].transform('mean')

        # Drop duplicates
        df.drop_duplicates(subset=['participantId'], inplace=True)

        print(df.head())
        print('----------------\n')

# Sort by participantId
df_bm.sort_values(by=['participantId'], inplace=True)
df_h.sort_values(by=['participantId'], inplace=True)

# Rename value columns
df_bm.rename(columns={'value': 'bodyMass_kg'}, inplace=True)
df_h.rename(columns={'value': 'height_m'}, inplace=True)

# Merge on participantId

df_bmi = pd.merge(df_bm, df_h, on='participantId')

df_bmi['startDate'] = df_bmi['timestamp_x'].dt.date

df_bmi['bmi'] = df_bmi['bodyMass_kg'] / (df_bmi['height_m'] ** 2)

# drop unnecessary columns and duplicates
df_bmi.drop(columns=['timestamp_x', 'timestamp_y'], inplace=True)
df_bmi.drop_duplicates(subset=['participantId'], inplace=True)

print(df_bmi.head())


                          participantId                 timestamp       value
0  11b2bf4d-6020-4a86-81bd-237c5616c649 2016-08-24 23:56:14+00:00   97.976000
1  9cbbb597-30c0-4f48-aebd-7fe58c627cf6 2016-03-16 02:33:20+00:00   63.503000
2  941e9fa4-6ba5-41f5-9bcc-805849d751b8 2017-01-01 14:07:00+00:00   96.666000
3  41c683b0-f6c4-4a6a-8dd2-4b9a3dd30a32 2016-03-30 02:24:59+00:00  112.846333
4  35fbae45-0f21-45ea-8ed7-8d3361a29b86 2016-07-18 17:45:38+00:00   68.690083
----------------

                          participantId                 timestamp   value
0  11b2bf4d-6020-4a86-81bd-237c5616c649 2016-08-24 23:56:14+00:00  1.6256
1  9cbbb597-30c0-4f48-aebd-7fe58c627cf6 2016-03-16 02:33:20+00:00  1.6510
2  941e9fa4-6ba5-41f5-9bcc-805849d751b8 2017-01-01 14:07:00+00:00  1.8288
3  1cba6631-3fb8-4c18-904f-ab56fe82833d 2016-03-07 05:54:00+00:00  1.5494
4  eb2c4ff0-54e2-4c34-b73d-003e3e0427ad 2016-04-02 07:17:47+00:00  1.7018
----------------

                          participantId  bodyMass_kg

# Heart Rate

In [3]:
import pandas as pd
from datetime import time

df_hr = pd.read_csv(folder_path + 'heart-rate.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])

# Convert startTime and endTime to datetime
df_hr['startTime'] = pd.to_datetime(df_hr['startTime'])
df_hr['endTime'] = pd.to_datetime(df_hr['endTime'])
df_hr['startDate'] = df_hr['startTime'].dt.date

# Remove rows where participantId, startTime, and endTime are duplicated
df_hr.drop_duplicates(subset=['participantId', 'startTime', 'endTime'], inplace=True)

# Initialize columns for morning, afternoon, and evening/night
df_hr['morning_hr'] = 0
df_hr['afternoon_hr'] = 0
df_hr['evening_hr'] = 0
df_hr['night_hr'] = 0

morning_end = time(12, 0)
afternoon_end = time(18, 0)
evening_end = time(23, 59)
night_end = time(6, 0)

# Classify each heart rate entry
for index, row in df_hr.iterrows():
    start_time = row['startTime'].time()
    if start_time < morning_end and start_time > night_end:
        df_hr.at[index, 'morning_hr'] = row['value']
    elif start_time < afternoon_end:
        df_hr.at[index, 'afternoon_hr'] = row['value']
    elif start_time < evening_end:
        df_hr.at[index, 'evening_hr'] = row['value']
    else:
        df_hr.at[index, 'night_hr'] = row['value']

# Calculate mean, max, and min heart rate for each period
df_hr_aggregated = df_hr.groupby(['participantId', 'startDate']).agg(
    mean_hr_morning=('morning_hr', 'mean'),
    max_hr_morning=('morning_hr', 'max'),
    min_hr_morning=('morning_hr', 'min'),
    mean_hr_afternoon=('afternoon_hr', 'mean'),
    max_hr_afternoon=('afternoon_hr', 'max'),
    min_hr_afternoon=('afternoon_hr', 'min'),
    mean_hr_evening=('evening_hr', 'mean'),
    max_hr_evening=('evening_hr', 'max'),
    min_hr_evening=('evening_hr', 'min'),
    mean_hr_night=('night_hr', 'mean'),
    max_hr_night=('night_hr', 'max'),
    min_hr_night=('night_hr', 'min')
).reset_index()

# Merge the aggregated values back into the original summary
df_hr_summary = df_hr.drop(columns=['morning_hr', 'afternoon_hr', 'evening_hr', 'night_hr']).drop_duplicates(subset=['participantId', 'startDate'])
df_hr_final = pd.merge(df_hr_summary, df_hr_aggregated, on=['participantId', 'startDate'], how='left')

# Drop unnecessary columns and rename
df_hr_final.drop(columns=['timestamp', 'startTime', 'endTime', 'value'], inplace=True)


print(df_hr_final.head())


                          participantId   startDate  mean_hr_morning  \
0  06bc6ebb-a233-469f-8091-90256f656b1b  2016-03-13           0.0721   
1  2214f4fd-1ae0-4804-8663-b01c5f6d142c  2016-03-02           0.0000   
2  b2571643-4aec-492f-bc7f-6f23c7fe239a  2018-10-03           0.6535   
3  b1406c4e-e6ac-4297-a9e4-335ca5ef04de  2019-01-24           0.0000   
4  b1406c4e-e6ac-4297-a9e4-335ca5ef04de  2019-01-23           0.0000   

   max_hr_morning  min_hr_morning  mean_hr_afternoon  max_hr_afternoon  \
0           1.233             0.0           1.078358             2.133   
1           0.000             0.0           1.215449             1.783   
2           1.533             0.0           0.609571             1.250   
3           0.000             0.0           1.136000             1.400   
4           0.000             0.0           0.000000             0.000   

   min_hr_afternoon  mean_hr_evening  max_hr_evening  min_hr_evening  \
0               0.0         0.022624           1.6

# Step Count

In [4]:

df_sc = pd.read_csv(folder_path + 'step-count.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])

# get total step count per day for each participant

df_sc['startTime'] = pd.to_datetime(df_sc['startTime'], utc=True)
df_sc['endTime'] = pd.to_datetime(df_sc['endTime'], utc=True)
df_sc['startDate'] = df_sc['startTime'].dt.date

df_sc['duration'] = df_sc['endTime'] - df_sc['startTime']
df_sc['duration'] = df_sc['duration'].dt.total_seconds()

df_sc['totalSteps'] = df_sc.groupby(['participantId', 'startDate'])['value'].transform('sum')
df_sc['totalSteps'] = df_sc['totalSteps'].astype(float)

# Get the total duration per day for each participant
df_sc['stepsTotalDuration'] = df_sc.groupby(['participantId', 'startDate'])['duration'].transform('sum')

df_sc.drop(columns=['duration'], inplace=True)

morning_end = time(12, 0)
afternoon_end = time(18, 0)
evening_end = time(23, 59)
night_end = time(6, 0)

df_sc['morningSteps'] = 0
df_sc['afternoonSteps'] = 0
df_sc['eveningSteps'] = 0
df_sc['nightSteps'] = 0

for index, row in df_sc.iterrows():
    start_time = row['startTime'].time()
    if start_time < morning_end and start_time > night_end:
        df_sc.at[index, 'morningSteps'] = row['value']
    elif start_time < afternoon_end:
        df_sc.at[index, 'afternoonSteps'] = row['value']
    elif start_time < evening_end:
        df_sc.at[index, 'eveningSteps'] = row['value']
    else:
        df_sc.at[index, 'nightSteps'] = row['value']

df_summary = df_sc.groupby(['participantId', 'startDate']).agg(
    morningStepsTotal=('morningSteps', 'sum'),
    afternoonStepsTotal = ('afternoonSteps', 'sum'),
    eveningStepsTotal =('eveningSteps', 'sum'),
    nightStepsTotal=('nightSteps', 'sum')
).reset_index()

df_sc = pd.merge(df_sc, df_summary, how='left', on=['participantId', 'startDate'])

# drop duplicates of participantId and startDate

df_sc.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

df_sc.drop(columns=['value', 'startTime', 'endTime', 'timestamp', 'morningSteps', 'afternoonSteps', 'eveningSteps', 'nightSteps'], inplace=True)

print(df_sc.head())

                            participantId   startDate  totalSteps  \
0    1f649060-680a-4c80-a551-be38ce46cb94  2016-03-22     10017.0   
21   1f649060-680a-4c80-a551-be38ce46cb94  2016-03-23     14752.0   
91   1f649060-680a-4c80-a551-be38ce46cb94  2016-03-24     15744.0   
116  1f649060-680a-4c80-a551-be38ce46cb94  2016-03-29     14477.0   
127  1f649060-680a-4c80-a551-be38ce46cb94  2016-03-30     12176.0   

     stepsTotalDuration  morningStepsTotal  afternoonStepsTotal  \
0               10939.0                  0                 4451   
21              13836.0                  0                 9184   
91              13291.0                 20                10155   
116             15828.0                118                 9103   
127             12559.0                  0                 7863   

     eveningStepsTotal  nightStepsTotal  
0                 5529               37  
21                5568                0  
91                5569                0  
116           

# Distance Walking/Running

In [5]:

df_dwr = pd.read_csv(folder_path + 'distance-walking-running.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp'])

# Convert timestamp to datetime and get date

df_dwr['startTime'] = pd.to_datetime(df_dwr['startTime'], utc=True)
df_dwr['endTime'] = pd.to_datetime(df_dwr['endTime'], utc=True)
df_dwr['timestamp'] = pd.to_datetime(df_dwr['timestamp'], utc=True)

df_dwr['startDate'] = df_dwr['startTime'].dt.date

# Get the total distance walked/ran per day for each participant
df_dwr['totalDistance'] = df_dwr.groupby(['participantId', 'startDate'])['value'].transform('sum')

# Get the duration
df_dwr['duration'] = df_dwr['endTime'] - df_dwr['startTime']
df_dwr['duration'] = df_dwr['duration'].dt.total_seconds()

df_dwr['dwrTotalDuration'] = df_dwr.groupby(['participantId', 'startDate'])['duration'].transform('sum')

df_dwr['morningDist'] = 0
df_dwr['afternoonDist'] = 0
df_dwr['eveningDist'] = 0
df_dwr['nightDist'] = 0

morning_end = time(12, 0)
afternoon_end = time(18, 0)
evening_end = time(23, 59)
night_end = time(6, 0)

for index, row in df_dwr.iterrows():
    start_time = row['startTime'].time()
    if start_time < morning_end and start_time > night_end:
        df_dwr.at[index, 'morningDist'] = row['value']
    elif start_time < afternoon_end:
        df_dwr.at[index, 'afternoonDist'] = row['value']
    elif start_time < evening_end:
        df_dwr.at[index, 'eveningtDist'] = row['value']
    else:
        df_dwr.at[index, 'nightDist'] = row['value']

df_summary = df_dwr.groupby(['participantId', 'startDate']).agg(
    morningDistance=('morningDist', 'sum'),
    afternoonDistance=('afternoonDist', 'sum'),
    eveningDistance=('eveningDist', 'sum'),
    nightDistance=('nightDist', 'sum')
).reset_index()

df_dwr = pd.merge(df_dwr, df_summary, how='left', on=['participantId', 'startDate'])
df_dwr.drop(columns=['value', 'startTime', 'endTime', 'timestamp', 'morningDist', 'afternoonDist', 'eveningDist', 'nightDist'], inplace=True)


# drop duplicates of participantId and startDate
df_dwr.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

print(df_dwr)

                               participantId   startDate  totalDistance  \
0       bfc93c66-90b4-4264-be63-b57c8b1a148f  2016-03-03       7140.476   
48      35fbae45-0f21-45ea-8ed7-8d3361a29b86  2016-06-16     103099.934   
49      41ec0944-d752-49c8-b6bd-b663abfd8f82  2016-12-06      34812.350   
50      4fee96d8-a09f-40f7-b8aa-132b062ae0a2  2016-05-23      14423.381   
55      3c6f9ed0-deaf-4996-8e26-e41787a525dd  2016-03-21         61.143   
...                                      ...         ...            ...   
943946  bb3b8938-47f4-42b5-a5f1-4b29fe5f5b99  2019-06-18         84.024   
943955  bb3b8938-47f4-42b5-a5f1-4b29fe5f5b99  2019-06-19        101.200   
943959  2e8e2771-f234-45a3-b4e6-6d197bb2c417  2019-06-20       2010.610   
943966  2e8e2771-f234-45a3-b4e6-6d197bb2c417  2019-06-21         42.040   
944043  d5d4940d-a826-44ab-a6a1-69fccbe1f80e  2019-06-26        808.810   

        duration  dwrTotalDuration  eveningtDist  morningDistance  \
0          360.0           125

# AM check-in

In [6]:

df_amch = pd.read_csv(folder_path + 'am-checkin.csv', usecols=['participantId', 'timestamp', 'AMCH1', 'AMCH2', 'AMCH2A','AMCH3','AMCH3A','AMCH4','AMCH5'], parse_dates=['timestamp'])

# fill NaN values of AMCH2A and AMCH3A with 0
df_amch['AMCH2A'] = df_amch['AMCH2A'].fillna(0)
df_amch['AMCH3A'] = df_amch['AMCH3A'].fillna(0)

df_amch['timestamp'] = pd.to_datetime(df_amch['timestamp'], utc=True)

df_amch['startDate'] = df_amch['timestamp'].dt.date

df_amch.drop(columns=['timestamp'], inplace=True)

# drop amch1, amch4
df_amch.drop(columns=['AMCH1', 'AMCH4'], inplace=True)

# for amch2, change true to 1 and false to 0
df_amch['AMCH2'] = df_amch['AMCH2'].map({True: 1, False: 0})

# drop any row with NaN values
df_amch = df_amch.dropna()

print(df_amch.head())


                          participantId  AMCH2  AMCH2A  AMCH3  AMCH3A  AMCH5  \
0  524909f5-77f5-4491-a19d-6893e8d37063    0.0     0.0    1.0     5.0  360.0   
1  32c768b8-24b8-4bdc-9cd3-1b9ec1856dfd    0.0     0.0    3.0     6.0  480.0   
4  aea751f7-98fb-4599-8ebd-927f438b853e    0.0     0.0    2.0    61.0  421.0   
5  40fc17cb-c891-42a6-b87e-851f9c5e7eda    1.0    61.0    5.0   121.0  301.0   
6  f1d27333-2d9f-4d60-9b7b-3aecbcbd67e1    0.0     0.0    3.0    12.0  330.0   

    startDate  
0  2016-03-05  
1  2016-09-26  
4  2016-05-13  
5  2016-09-26  
6  2016-06-16  


# PM check-in

In [7]:

df_pmch = pd.read_csv(folder_path + 'pm-checkin.csv', usecols=['participantId', 'timestamp', 'alcohol', 'caffeine', 'NapCount','PMCH1','PMCH2A','PMCH3'], parse_dates=['timestamp'])

df_pmch['timestamp'] = pd.to_datetime(df_pmch['timestamp'], utc=True)

df_pmch['startDate'] = df_pmch['timestamp'].dt.date

df_pmch['PMCH2A'] = df_pmch['PMCH2A'].fillna(0)
df_pmch['NapCount'] = df_pmch['NapCount'].fillna(0)
df_pmch['alcohol'] = df_pmch['alcohol'].fillna(0)
df_pmch['caffeine'] = df_pmch['caffeine'].fillna(0)

df_pmch['medication'] = 0

def check_medication(value):
    return '100' in str(value).split(',')

df_pmch['medication'] = df_pmch.apply(lambda row: '1' if check_medication(row['PMCH3']) else row['medication'], axis=1)

df_pmch.drop(columns=['timestamp'], inplace=True)
df_pmch.drop(columns=['PMCH3'], inplace=True)
df_pmch = df_pmch.dropna()

print(df_pmch.head())



                          participantId  NapCount  PMCH1  PMCH2A  alcohol  \
0  d0108687-0096-4fc1-a065-7e6b5ad1f281       0.0    3.0     0.0        0   
1  c64cd0ef-800a-416a-b1ce-42b93f1a8e38       0.0    3.0     0.0        0   
2  4aed0e57-1cb6-4850-90ad-144de519d1b4       0.0    3.0     0.0        0   
3  c39e7d65-9f09-4b35-899c-9aa40f98fca6       0.0    3.0     0.0        0   
4  6da7e848-daaa-410c-a83f-35b63498595c       0.0    3.0     0.0        0   

   caffeine   startDate medication  
0         0  2016-03-03          0  
1         6  2016-03-03          1  
2         6  2016-03-03          0  
3         4  2016-11-04          1  
4         2  2016-03-02          0  


# Sleep Quality

In [8]:
df_sq = pd.read_csv(folder_path + 'sleep-quality-checker.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])

df_sq['timestamp'] = pd.to_datetime(df_sq['timestamp'], utc=True)

df_sq['startDate'] = df_sq['timestamp'].dt.date

# drop duplicates
df_sq.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

# drop unnecessary columns
df_sq.drop(columns=['timestamp'], inplace=True)

df_sq.rename(columns={'value': 'ssq_score'}, inplace=True)

print(df_sq.head())


                          participantId  ssq_score   startDate
0  b4ebf7dd-4e30-4f7b-8ee8-5493a19c8c9f          4  2016-09-27
1  c3993552-69cb-45e4-b18a-5e6eecefb825          4  2016-03-07
2  78f60bd3-34f3-489e-a352-f9df564641c3          4  2016-03-05
3  9da1a89a-2145-4cca-b356-7b58aa7be8b0          4  2016-09-27
4  4aad9dbe-dd9e-4832-a198-3bd563457124          4  2016-03-03


# Survey Responses

In [9]:
import pandas as pd

# Read in data from surveys
# For each unique participnat in subset_dfs, check if they have answered both surveys. If so, copy the values to each row for that participant
# Note see above code

folder_path = 'c:/Users/aoife/Documents/Project/DataTables/'


df_about_me = pd.read_csv(folder_path + 'about-me.csv', usecols=['participantId', 'timestamp', 'alcohol', 'basic_expenses', 'caffeine', 'daily_activities', 'daily_activities', 'daily_smoking', 'education', 'flexible_work_hours', 'gender', 'good_life', 'hispanic', 'income', 'marital', 'race', 'smoking_status', 'menopause', 'recent_births', 'current_pregnant', 'work_schedule'], parse_dates=['timestamp'])

df_sleep_habits = pd.read_csv(folder_path + 'sleep-habits.csv', usecols=['participantId', 'timestamp', 'alarm_dependency', 'driving_sleepy', 'falling_asleep', 'morning_person', 'nap_duration', 'sleep_lost', 'sleep_needed', 'sleep_partner', 'sleep_time_workday', 'sleep_time_weekend', 'wake_up_choices', 'wake_ups', 'weekly_naps', 'what_wakes_you'], parse_dates=['timestamp'])



In [10]:
# rename the alcolhol column in about_me and sleep_assessment to be more detailed

df_about_me.rename(columns={'alcohol': 'alcohol_consumption'}, inplace=True)

# Convert timestamp to datetime
df_about_me['timestamp'] = pd.to_datetime(df_about_me['timestamp'], utc=True)
df_sleep_habits['timestamp'] = pd.to_datetime(df_sleep_habits['timestamp'], utc=True)

# Change the timestamp to only contain the date
df_about_me['date'] = df_about_me['timestamp'].dt.date
df_sleep_habits['date'] = df_sleep_habits['timestamp'].dt.date


In [11]:
# Merge survey dfs

surveys_list = [df_about_me, df_sleep_habits]

# Merge the DataFrames
df_surveys = pd.merge(surveys_list[0], surveys_list[1], on=['participantId', 'date'], how='outer', suffixes=('', '_y'))
#df_surveys = pd.merge(df_surveys, surveys_list[2], on=['participantId', 'date'], how='outer', suffixes=('', '_y'))


In [12]:
# Remove duplicates
df_surveys.drop_duplicates(subset=['participantId'], inplace=True)

# For the 'menopause' column, replace NaN with 3
df_surveys['menopause'] = df_surveys['menopause'].fillna(3)

# For the recent_births column, replace NaN with 4
df_surveys['recent_births'] = df_surveys['recent_births'].fillna(4)

# Replace current_pregnant NaN with 0
df_surveys['current_pregnant'] = df_surveys['current_pregnant'].fillna(0)

# replace driving_sleepy NaN with 6
df_surveys['driving_sleepy'] = df_surveys['driving_sleepy'].fillna(6)

# replace falling_asleep NaN with 0
df_surveys['falling_asleep'] = df_surveys['falling_asleep'].fillna(0)

# replace morning_person NaN with 3
df_surveys['morning_person'] = df_surveys['morning_person'].fillna(3)

# replace nap_duration NaN with 6
df_surveys['nap_duration'] = df_surveys['nap_duration'].fillna(6)

# replace sleep_lost NaN with 0
df_surveys['sleep_lost'] = df_surveys['sleep_lost'].fillna(0)

# replace what_wakes_you NaN with 13
df_surveys['what_wakes_you'] = df_surveys['what_wakes_you'].fillna(13)

# Remove unnecessary columns

df_surveys.drop(columns=['timestamp', 'date', 'timestamp_y'], inplace=True)

In [13]:
def join_multiple_race(row):
    if ',' in str(row):
        return 6
    return row

df_surveys['hispanic'] = df_surveys['hispanic'].apply(join_multiple_race)
df_surveys['race'] = df_surveys['race'].apply(join_multiple_race)

def join_multiple_sleep_partner(row):
    if ',' in str(row):
        return 6
    return row

df_surveys['sleep_partner'] = df_surveys['sleep_partner'].apply(join_multiple_sleep_partner)

# create new columns for wakeup reasons

df_surveys['noise_light'] = 0
df_surveys['stress_thinking'] = 0
df_surveys['other_person'] = 0
df_surveys['pain_discomfort'] = 0
df_surveys['nightmares'] = 0
df_surveys['bathroom_urges'] = 0
df_surveys['other_reasons'] = 0

def check_wakeup_reason(row, number):
    if number == 1:
        df_surveys.loc[row.name, 'noise_light'] = 1
    elif number == 2:
        df_surveys.loc[row.name, 'stress_thinking'] = 1
    elif number == 3:
        df_surveys.loc[row.name, 'other_person'] = 1
    elif number == 4:
        df_surveys.loc[row.name, 'pain_discomfort'] = 1
    elif number == 5:
        df_surveys.loc[row.name, 'nightmares'] = 1
    elif number == 6:
        df_surveys.loc[row.name, 'bathroom_urges'] = 1
    else:
        df_surveys.loc[row.name, 'other_reasons'] = 1


for i, row in df_surveys.iterrows():
    if ',' in str(row['what_wakes_you']):
        nums = map(int, row['what_wakes_you'].split(','))
        for number in nums:
            check_wakeup_reason(row, number)
    else:
        check_wakeup_reason(row, int(row['what_wakes_you']))

# Remove what_wakes_you
df_surveys.drop(columns=['what_wakes_you'], inplace=True)

print(df_surveys.head())

                          participantId  alcohol_consumption  basic_expenses  \
0  0c82c9d1-25ba-4cb2-95df-f79fca0b8464                  1.0             3.0   
1  20a71d11-3d78-4ee0-a172-5dd8f7e33bd2                  0.0             4.0   
2  f5aac809-fd16-4733-83ee-0991eaf7036f                  0.0             3.0   
3  5ceb42a9-cd99-4ba8-93d8-4fc4a5de5a4f                  2.0             4.0   
4  80bfb0f6-601c-47ed-9538-bedb8eb6c69f                  1.0             2.0   

   caffeine  daily_activities  daily_smoking  education  flexible_work_hours  \
0       4.0               1.0            1.0        4.0                  2.0   
1       5.0               1.0            3.0        4.0                  1.0   
2       3.0               1.0            3.0        4.0                  2.0   
3       1.0               1.0            3.0        5.0                  2.0   
4       3.0               4.0            3.0        5.0                  2.0   

   gender  good_life  ... wake_up_choi

# Pulling Everything Together

1. Add each dataframe to a list
2. Gather a list of unique participants
3. Choose a number of unique participants, and merge their data across each dataframe
4. Continue for all participants
5. Remove NaN values
6. Concatenate each new dataframe

In [14]:
# merge function

def merge_dfs(dfs_list, participants):
    merged_df = None
    for df in dfs_list:
        # Filter dataframe for participants we currently work with
        curr_df = df[df['participantId'].isin(participants)]
        if merged_df is None:
            merged_df = curr_df
        else:
            merged_df = pd.merge(merged_df, curr_df, on=['participantId', 'startDate'], how='outer', suffixes=('', '_y'))
    return merged_df


In [15]:

def get_separate_tables(num_participants):
    
    # Create a list of DataFrames
    dfs_list = [df_bmi, df_hr_final, df_sc, df_dwr, df_amch, df_sq]	

    # df_pmch not included because of lack of data

    # Create a list of all participants
    participants = set(dfs_list[0]['participantId'].unique())

    # Sort the list of participants alphabetically
    participants = sorted(list(participants))

    result_dfs = []

    while participants:
        # Get the next group of participants
        next_group = participants[:num_participants]

        # Merge the DataFrames for the next group of participants
        merge_df = merge_dfs(dfs_list, next_group)

        print(merge_df.head())

        # Add the merged DataFrame to the list of results
        result_dfs.append(merge_df)

        # Remove the participants that were just used
        participants = participants[num_participants:]

    return result_dfs


In [16]:
# Call get_separate_tables() and check the resulting dfs

subset_dfs = get_separate_tables(20)

for i, df in enumerate(subset_dfs):
    print(f"DataFrame {i}:")
    print(df.head())
    print(f"Number of rows: {len(df)}")
    print("\n" + "-"*30 + "\n")


                          participantId  bodyMass_kg  height_m   startDate  \
0  00a55fb5-da33-4e2e-ae61-28f589fcc174       51.256    1.5748  2016-05-12   
1  00d6d2ee-ccea-45c7-9772-b19fd9bef2bf       74.389    1.9050  2018-04-20   
2  00fd4039-9b5e-4bbb-8295-4983a3f58371       59.874    1.6510  2018-08-15   
3  0113e483-0fc8-4892-a4fc-0b2f3820dde3       68.039    1.7018  2019-03-06   
4  013d82d7-b3cc-4007-b00c-4d1d75bab9dd       76.204    1.6002  2018-02-21   

         bmi  mean_hr_morning  max_hr_morning  min_hr_morning  \
0  20.667783          0.18334           1.617             0.0   
1  20.498343              NaN             NaN             NaN   
2  21.965653          0.51139           1.283             0.0   
3  23.493130          0.00000           0.000             0.0   
4  29.759747              NaN             NaN             NaN   

   mean_hr_afternoon  max_hr_afternoon  ...  morningDistance  \
0           1.013213             1.783  ...          1123.06   
1           

In [17]:
# For each unique participant, check if they have a value for weight, height, and bmi. If so, copy values to each row for that participant

for df in subset_dfs:

    # Loop through participants
    for participant in df['participantId'].unique():
        # Find the index of the row containing non NaN values for weight, height, and bmi
        non_nan_index = df[(df['participantId'] == participant) & (~df['bodyMass_kg'].isnull()) & (~df['height_m'].isnull()) & (~df['bmi'].isnull())].index

        # If there is a row with non NaN values, copy the values to all rows for that participant
        if len(non_nan_index) > 0:
            non_nan_index = non_nan_index[0]
            non_nan_row = df.loc[non_nan_index]
            df.loc[df['participantId'] == participant, 'bodyMass_kg'] = non_nan_row['bodyMass_kg']
            df.loc[df['participantId'] == participant, 'height_m'] = non_nan_row['height_m']
            df.loc[df['participantId'] == participant, 'bmi'] = non_nan_row['bmi']


In [18]:
# Concatenate the DataFrames in subset_dfs into a single DataFrame

concatenated_df = pd.concat(subset_dfs, ignore_index=True)

print(concatenated_df.head())

# print total number of rows in concatenated_df
print(f"Total number of rows in concatenated_df: {len(concatenated_df)}")


                          participantId  bodyMass_kg  height_m   startDate  \
0  00a55fb5-da33-4e2e-ae61-28f589fcc174       51.256    1.5748  2016-05-12   
1  00d6d2ee-ccea-45c7-9772-b19fd9bef2bf       74.389    1.9050  2018-04-20   
2  00fd4039-9b5e-4bbb-8295-4983a3f58371       59.874    1.6510  2018-08-15   
3  0113e483-0fc8-4892-a4fc-0b2f3820dde3       68.039    1.7018  2019-03-06   
4  013d82d7-b3cc-4007-b00c-4d1d75bab9dd       76.204    1.6002  2018-02-21   

         bmi  mean_hr_morning  max_hr_morning  min_hr_morning  \
0  20.667783          0.18334           1.617             0.0   
1  20.498343              NaN             NaN             NaN   
2  21.965653          0.51139           1.283             0.0   
3  23.493130          0.00000           0.000             0.0   
4  29.759747              NaN             NaN             NaN   

   mean_hr_afternoon  max_hr_afternoon  ...  morningDistance  \
0           1.013213             1.783  ...          1123.06   
1           

In [19]:
# Merge the surveys DataFrame with the concatenated DataFrame

activity_and_survey_df = pd.merge(concatenated_df, df_surveys, on=['participantId'], how='outer', suffixes=('', '_y'))

print(activity_and_survey_df.head())

# Print num rows
print(f"Number of rows in activity_and_survey_df before NaN removed: {len(activity_and_survey_df)}")

# Remove rows that have NaN values
#activity_and_survey_df = activity_and_survey_df.dropna()

#print(f"Number of rows in activity_and_survey_df after NaN removed: {len(activity_and_survey_df)}")

#print(activity_and_survey_df.head())

activity_and_survey_df.to_csv(save_path + 'timed_df.csv', index=False)


                          participantId  bodyMass_kg  height_m   startDate  \
0  00a55fb5-da33-4e2e-ae61-28f589fcc174       51.256    1.5748  2016-05-12   
1  00a55fb5-da33-4e2e-ae61-28f589fcc174       51.256    1.5748  2016-05-14   
2  00a55fb5-da33-4e2e-ae61-28f589fcc174       51.256    1.5748  2016-05-13   
3  00a55fb5-da33-4e2e-ae61-28f589fcc174       51.256    1.5748  2016-05-20   
4  00a55fb5-da33-4e2e-ae61-28f589fcc174       51.256    1.5748  2016-05-18   

         bmi  mean_hr_morning  max_hr_morning  min_hr_morning  \
0  20.667783          0.18334           1.617             0.0   
1  20.667783              NaN             NaN             NaN   
2  20.667783              NaN             NaN             NaN   
3  20.667783              NaN             NaN             NaN   
4  20.667783              NaN             NaN             NaN   

   mean_hr_afternoon  max_hr_afternoon  ...  wake_up_choices  wake_ups  \
0           1.013213             1.783  ...              5.0      

In [20]:
# remove rows where startDate and participant Id are duplicated

activity_and_survey_df.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

In [21]:
# Drop rows with NaN values
activity_and_survey_df = activity_and_survey_df.dropna()

# Save the final DataFrame to a CSV file
activity_and_survey_df.to_csv(save_path + 'final_timed_data.csv', index=False)

In [22]:
print(activity_and_survey_df.head(10))

                             participantId  bodyMass_kg  height_m   startDate  \
3103  38d8d1df-cf8e-454b-95c7-841d84820635    82.529504    1.8796  2017-11-05   
3124  38d8d1df-cf8e-454b-95c7-841d84820635    82.529504    1.8796  2018-06-18   
3914  3e6dce9a-31f9-42a4-bd68-cb040d4613c2    93.914800    1.9304  2018-04-30   
5902  55d49397-8aae-4c41-be90-a892217a9d1d   111.130000    1.6510  2018-11-03   
7102  696ed394-0da4-4a36-8b02-e59e68747a3d    70.995970    1.8542  2016-10-11   
7292  696ed394-0da4-4a36-8b02-e59e68747a3d    70.995970    1.8542  2017-02-03   
7297  696ed394-0da4-4a36-8b02-e59e68747a3d    70.995970    1.8542  2017-02-08   
7418  696ed394-0da4-4a36-8b02-e59e68747a3d    70.995970    1.8542  2017-06-30   
7429  696ed394-0da4-4a36-8b02-e59e68747a3d    70.995970    1.8542  2017-07-11   
7439  696ed394-0da4-4a36-8b02-e59e68747a3d    70.995970    1.8542  2017-07-21   

            bmi  mean_hr_morning  max_hr_morning  min_hr_morning  \
3103  23.360297              0.0        